# Plan
- create pf 6.0 manifest
- create file of known breakpoints
- create lists of positive samples
- create list of negative samples
- calc_proportion_faceaway function
- calc_breakpoint_spanning_reads function
- calc_coverage_change function

In [1]:
%run _standard_imports.ipynb


python 3.4.5 |Anaconda 2.2.0 (64-bit)| (default, Jul  2 2016, 17:47:47) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
numpy 1.11.1
scipy 0.18.0
pandas 0.15.2
numexpr 2.3.1
pysam 0.8.3
pysamstats 0.23
petl 1.1.0
petlx 1.0.3
vcf 0.6.7
vcfnp 2.3.0.dev0
h5py 2.4.0
tables 3.1.1


# Setup

In [143]:
genome_fn = '/lustre/scratch116/malaria/pfalciparum/resources/Pfalciparum.genome.fasta'
gff_fn = '/lustre/scratch116/malaria/pfalciparum/resources/snpEff/data/Pfalciparum_GeneDB_Aug2015/genes.gff'

scratch_dir = '/lustre/scratch109/malaria/rp7/data/methods-dev/analysis/20160926_pf_60_duplications'
results_dir = '/nfs/team112_internal/rp7/data/methods-dev/analysis/20160926_pf_60_duplications'

tandem_dup_breakpoints_fn = "%s/tandem_dup_breakpoints.txt" % results_dir
dup_trpinv_dup_breakpoints_fn = "%s/dup_trpinv_dup_breakpoints.txt" % results_dir
pf_6_0_bams_fn = "%s/pf_6_0_bams.txt" % results_dir

dup_results_hrp_list_fn = "%s/dup_results_hrp_list.txt" % results_dir
dup_results_6_0_fn = "%s/dup_results_6_0.txt" % results_dir
# dup_trpinv_dup_results_fn = "%s/dup_trpinv_dup_results.txt" % results_dir


positive_controls = collections.OrderedDict()
positive_controls['17kb from plasmepsin I'] = ['PH0243-C', 'PH0247-C', 'PH0484-C']
positive_controls['17kb from plasmepsin IV'] = ['PH0243-C', 'PH0247-C', 'PH0484-C']
positive_controls['80kb from plasmepsin IV'] = ['PH0906-C', 'PH0912-C']
positive_controls['18kb around MDR1'] = ['PH0254-C']
positive_controls['96kb around MDR1'] = ['PH0233-C']
positive_controls['57kb dup-trpinv-dup MDR1'] = ['PH0267-C']
positive_controls['82kb around MDR1'] = ['PC0080-C']
positive_controls['169kb around MDR1'] = ['QC0156-C']
positive_controls['94kb around MDR1'] = ['PD0943-C']
positive_controls['199kb around MDR1'] = ['PG0116-C']

negative_controls = []


In [ ]:
!mkdir -p {scratch_dir}
!mkdir -p {results_dir}


In [162]:
# Create the tandem duplication breakpoints file. Long term this will be a growing list of known breakpoints.
fo = open(tandem_dup_breakpoints_fn, 'w')

print(
    '''chrom\tfirst_breakpoint_start\tfirst_breakpoint_end\tsecond_breakpoint_start\tsecond_breakpoint_end\tstart_gene_id\tend_gene_id\tbreakpoint_name
Pf3D7_05_v3\t938329\t938357\t980012\t980040\tPF3D7_0522900\tPF3D7_0523000\t42kb around MDR1
Pf3D7_05_v3\t949514\t949527\t967253\t967266\tPF3D7_0522900\tPF3D7_0523000\t17.7kb around MDR1
Pf3D7_05_v3\t947790\t947800\t962444\t962454\tPF3D7_0522900\tPF3D7_0523000\t15kb around MDR1
Pf3D7_05_v3\t953961\t953982\t973008\t973034\tPF3D7_0523000\tPF3D7_0523100\t19kb around MDR1
Pf3D7_05_v3\t953961\t953982\t965409\t865428\tPF3D7_0523000\tPF3D7_0523100\t11kb around MDR1
Pf3D7_05_v3\t947968\t947986\t969783\t969812\tPF3D7_0523000\tPF3D7_0523200\t22kb around MDR1
Pf3D7_05_v3\t953961\t953982\t970215\t970252\tPF3D7_0523000\tPF3D7_0523100\t16kb around MDR1
Pf3D7_05_v3\t780906\t780927\t980012\t980040\tPF3D7_0518800\tPF3D7_0523600\t199kb around MDR1
Pf3D7_05_v3\t870473\t870502\t964628\t964646\tPF3D7_0521300\tPF3D7_0523100\t94kb around MDR1
Pf3D7_05_v3\t795494\t795527\t964505\t964540\tPF3D7_0519300\tPF3D7_0519300\t169kb around MDR1
Pf3D7_05_v3\t888324\t888349\t970215\t970252\tPF3D7_0521900\tPF3D7_0523200\t82kb around MDR1
Pf3D7_05_v3\t868667\t868699\t964505\t964540\tPF3D7_0521200\tPF3D7_0523100\t96kb around MDR1
Pf3D7_05_v3\t946696\t946717\t964505\t964540\tPF3D7_0522900\tPF3D7_0523100\t18kb around MDR1
Pf3D7_05_v3\t946696\t946717\t970215\t970252\tPF3D7_0522900\tPF3D7_0523100\t24kb around MDR1
Pf3D7_14_v3\t283034\t283069\t300493\t300522\tPF3D7_1407800\tPF3D7_1409200\t17kb from plasmepsin IV
Pf3D7_14_v3\t283034\t283069\t362990\t363020\tPF3D7_1407800\tPF3D7_1409200\t80kb from plasmepsin IV
Pf3D7_14_v3\t289611\t289621\t298782\t298792\tPF3D7_1407800\tPF3D7_1409200\tplasmepsin I to plasmepsin III''',
    file=fo
)

fo.close()

In [126]:
# Create the dup-trp/inv-dup breakpoints file. Long term this will be a growing list of known breakpoints.
# See Fig 4e at http://www.nature.com/nrg/journal/v17/n4/full/nrg.2015.25.html for description of dup-trp/inv-dup
fo = open(dup_trpinv_dup_breakpoints_fn, 'w')

print(
    '''chrom\tfirst_breakpoint_start\tfirst_breakpoint_end\tsecond_breakpoint_start\tsecond_breakpoint_end\tthird_breakpoint_start\tthird_breakpoint_end\tfourth_breakpoint_start\tfourth_breakpoint_end\tfirst_gene_id\tsecond_gene_id\tthird_gene_id\tfourth_gene_id\tbreakpoint_name
Pf3D7_05_v3\t928340\t928359\t938911\t938930\t964505\t964532\t985396\t985423\tPF3D7_0522400\tPF3D7_0522600\tPF3D7_0523100\tPF3D7_0523700\t57kb dup-trpinv-dup MDR1''',
    file=fo
)

fo.close()

In [161]:
980012 - 938329

41683

In [68]:
# Create file with path to bam and ox_code
!vrpipe-fileinfo --setup pf_60_mergelanes --metadata sample > {pf_6_0_bams_fn}

# Functions

In [55]:
def calc_proportion_faceaway(
    bam_fn='/lustre/scratch109/malaria/pfalciparum/output/4/5/7/8/44843/1_bam_merge/pe.1.bam', # PH0906-C
    chrom='Pf3D7_14_v3',
    start_region=[282969, 283269], # 2-element list containing start and end positions (1-based) of region to right of first breakpoint in which to search for faceaway reads
    end_region=[362690, 362990], # 2-element list containing start and end positions (1-based) of region to left of second breakpoint in which to search for faceaway reads
):
    samfile = pysam.AlignmentFile(bam_fn, "rb")
    num_faceaway = 0
    num_reads = 0
    iter = samfile.fetch(chrom, start_region[0], start_region[1])
    for x in iter:
        num_reads += 1
        if (x.is_paired and x.is_reverse and (not x.mate_is_reverse) and x.mpos >= end_region[0] and x.mpos <= end_region[1]):
            num_faceaway += 1
    
    if num_reads > 0:
        proportion_faceaway = num_faceaway/num_reads
    else:
        proportion_faceaway = float(np.nan)
    
    return(proportion_faceaway, num_faceaway, num_reads)

calc_proportion_faceaway('/lustre/scratch116/malaria/pfalciparum/output/7/2/c/1/30749/2_gatk_print_reads_gatk3_v2/8668_7_16.pe.markdup.recal.bam')

(0.03680981595092025, 6, 163)

In [121]:
# This looks for pairs of reads mapping in the same direction (indicative of an inversion)

def calc_proportion_same_direction(
    bam_fn='/lustre/scratch109/malaria/pfalciparum/output/4/f/4/3/44301/1_bam_merge/pe.1.bam', # PH0267-C
    chrom='Pf3D7_05_v3',
    start_region=[928240, 928840], # 2-element list containing start and end positions (1-based) of region to left of first breakpoint in which to search for faceaway reads
    end_region=[938811, 939411], # 2-element list containing start and end positions (1-based) of region to left of second breakpoint in which to search for faceaway reads
    direction='reverse' # could also be forward
):
    samfile = pysam.AlignmentFile(bam_fn, "rb")
    num_same_direction = 0
    num_reads = 0
    iter = samfile.fetch(chrom, start_region[0], start_region[1])
    for x in iter:
        num_reads += 1
        if direction=='forward':
            if (x.is_paired and (not x.is_reverse) and (not x.mate_is_reverse) and x.mpos >= end_region[0] and x.mpos <= end_region[1]):
                num_same_direction += 1
        if direction=='reverse':
            if (x.is_paired and x.is_reverse and x.mate_is_reverse and x.mpos >= end_region[0] and x.mpos <= end_region[1]):
                num_same_direction += 1
    
    if num_reads > 0:
        proportion_same_direction = num_same_direction/num_reads
    else:
        proportion_same_direction = float(np.nan)
    
    return(proportion_same_direction, num_same_direction, num_reads)

calc_proportion_same_direction()

(0.05, 25, 500)

In [122]:
calc_proportion_same_direction(start_region=[964532-500, 964532+100], end_region=[985423-500, 985423+100], direction='forward')

(0.04772234273318872, 44, 922)

In [3]:
# This function will create a "mini-genome" containing three sequences: those from reference around each breakpoint,
# and a sequence containing breakpoint-spanning sequence. All reads from bam will be mapped to this mini-genome,
# and number of reads mapping to each sequence reported
def calc_breakpoint_spanning_reads(
    bam_fn,
    genome_fn = genome_fn,
    chrom,
    first_region, # 2-element list containing start and end positions (1-based) of common sequence at first breakpoint
    second_region, # 2-element list containing start and end positions (1-based) of common sequence at second breakpoint
    
):
    
    
    return(breakpoint_spanning_coverage, breakpoint_1_coverage, breakpoint_2_coverage, proportion_breakpoint_spanning)

In [ ]:
# This function will determine coverage within coding region of tested duplication, and also coverage of coding
# regions of genes immediately upstream and downstream of the region
def calc_coverage_change(
    bam_fn,
    chrom,
    duplication_start,
    duplication_end,
    first_gene, # ID of first gene within the duplication region
    last_gene, # ID of first gene within the duplication region
    gff_fn,
):
    
    
    return(
        coverage_duplication,
        coverage_upstream,
        covergae_downstream,
        coverage_change_upstream,
        coverage_change_downstream,
        coverage_change_both,
        expected_copy_number,
        is_correct_breakpoints,
    )

In [59]:
etl.fromtsv(breakpoints_fn).convertnumbers().display(index_header=True)

0|chrom,1|first_breakpoint_start,2|first_breakpoint_end,3|second_breakpoint_start,4|second_breakpoint_end,5|start_gene_id,6|end_gene_id,7|breakpoint_name
Pf3D7_14_v3,283034,283069,300493,300522,PF3D7_1407800,PF3D7_1409200,80kb from Plasmepsin IV


In [127]:
etl.fromtsv(dup_trpinv_dup_breakpoints_fn).convertnumbers().display(index_header=True)

0|chrom,1|first_breakpoint_start,2|first_breakpoint_end,3|second_breakpoint_start,4|second_breakpoint_end,5|third_breakpoint_start,6|third_breakpoint_end,7|fourth_breakpoint_start,8|fourth_breakpoint_end,9|first_gene_id,10|second_gene_id,11|third_gene_id,12|fourth_gene_id,13|breakpoint_name
Pf3D7_05_v3,928340,928359,938911,938930,964505,964532,985396,985423,PF3D7_0522400,PF3D7_0522600,PF3D7_0523100,PF3D7_0523700,57kb dup-trpinv-dup MDR1


In [137]:
def genotype_duplications(
    bam_fn = '/lustre/scratch116/malaria/pfalciparum/output/7/2/c/1/30749/2_gatk_print_reads_gatk3_v2/8668_7_16.pe.markdup.recal.bam',
    tandem_dup_breakpoints_fn = tandem_dup_breakpoints_fn,
    dup_trpinv_dup_breakpoints_fn = dup_trpinv_dup_breakpoints_fn,
    region_size = 600,
    region_offset = 100,
):
    tbl_tandem_dup_breakpoints = etl.fromtsv(tandem_dup_breakpoints_fn).convertnumbers()
    tbl_dup_trpinv_dup_breakpoints = etl.fromtsv(dup_trpinv_dup_breakpoints_fn).convertnumbers()
    
    genotype_results = collections.OrderedDict()
    genotype_results['faceaway'] = collections.OrderedDict()
    genotype_results['dup_trpinv_dup'] = collections.OrderedDict()
    
    for rec in tbl_tandem_dup_breakpoints.data():
        genotype_results['faceaway'][rec[7]] = calc_proportion_faceaway(
            bam_fn,
            chrom = rec[0],
            start_region = [
                rec[2] - region_offset,
                rec[2] - region_offset + region_size
            ],
            end_region = [
                rec[3] - region_size,
                rec[3],
            ]
        )[0]
    
    for rec in tbl_dup_trpinv_dup_breakpoints.data():
#         genotype_results['dup_trpinv_dup']["%s first" % rec[13]] = 0.0
        genotype_results['dup_trpinv_dup']["%s first" % rec[13]] = calc_proportion_same_direction(
#         print(calc_proportion_same_direction(
            bam_fn,
            chrom = rec[0],
            start_region = [
                rec[1] - region_offset,
                rec[1] - region_offset + region_size
            ],
            end_region = [
                rec[3] - region_offset,
                rec[3] - region_offset + region_size,
            ],
            direction = 'reverse'
        )[0]
        genotype_results['dup_trpinv_dup']["%s second" % rec[13]] = calc_proportion_same_direction(
            bam_fn,
            chrom = rec[0],
            start_region = [
                rec[6] - region_size + region_offset,
                rec[6] + region_offset
            ],
            end_region = [
                rec[8] - region_size + region_offset,
                rec[8] + region_offset,
            ],
            direction = 'forward'
        )[0]
    
    return(genotype_results)

genotype_duplications()

OrderedDict([('faceaway',
              OrderedDict([('199kb around MDR1', 0.0),
                           ('94kb around MDR1', 0.0),
                           ('169kb around MDR1', 0.0),
                           ('82kb around MDR1', 0.0),
                           ('96kb around MDR1', 0.0),
                           ('18kb around MDR1', 0.0),
                           ('17kb from plasmepsin IV', 0.0),
                           ('80kb from plasmepsin IV', 0.018691588785046728),
                           ('plasmepsin I to plasmepsin III',
                            0.003205128205128205)])),
             ('dup_trpinv_dup',
              OrderedDict([('57kb dup-trpinv-dup MDR1 first', 0.0),
                           ('57kb dup-trpinv-dup MDR1 second', 0.0)]))])

In [135]:
genotype_duplications('/lustre/scratch109/malaria/pfalciparum/output/4/f/4/3/44301/1_bam_merge/pe.1.bam')

OrderedDict([('faceaway',
              OrderedDict([('199kb around MDR1', 0.0),
                           ('94kb around MDR1', 0.0),
                           ('169kb around MDR1', 0.0),
                           ('82kb around MDR1', 0.0),
                           ('96kb around MDR1', 0.0),
                           ('18kb around MDR1', 0.0),
                           ('17kb from plasmepsin IV', 0.0),
                           ('80kb from plasmepsin IV', 0.0),
                           ('plasmepsin I to plasmepsin III', 0.0)])),
             ('dup_trpinv_dup',
              OrderedDict([('57kb dup-trpinv-dup MDR1 first', 0.05),
                           ('57kb dup-trpinv-dup MDR1 second',
                            0.04772234273318872)]))])

In [110]:
# This looks for pairs of reads mapping in the same direction (indicative of an inversion) in two separate locations
# Test with PH0267-C

def genotype_dup_trpinv_dups(
    bam_fn = '/lustre/scratch109/malaria/pfalciparum/output/4/f/4/3/44301/1_bam_merge/pe.1.bam',
    breakpoints_fn = dup_trpinv_dup_breakpoints_fn,
    reads_region_size = 500,
    reads_region_offset = 100,
):
    tbl_breakpoints = etl.fromtsv(breakpoints_fn).convertnumbers()
    
    genotype_results = collections.OrderedDict()
    genotype_results['same_direction'] = collections.OrderedDict()
    
    for rec in tbl_breakpoints.data():
        genotype_results['same_direction'][rec[7]] = calc_proportion_faceaway(
            bam_fn,
            chrom = rec[0],
            start_region = [
                rec[2] - faceaway_region_offset,
                rec[2] - faceaway_region_offset + faceaway_region_size
            ],
            end_region = [
                rec[3] - faceaway_region_size,
                rec[3],
            ]
        )[0]
    
    return(genotype_results)

genotype_tandem_dups()

OrderedDict([('faceaway',
              OrderedDict([('199kb around MDR1', 0.0),
                           ('94kb around MDR1', 0.0),
                           ('169kb around MDR1', 0.0),
                           ('82kb around MDR1', 0.0),
                           ('96kb around MDR1', 0.0),
                           ('18kb around MDR1', 0.0),
                           ('17kb from plasmepsin IV', 0.0),
                           ('80kb from plasmepsin IV', 0.03680981595092025),
                           ('plasmepsin I to plasmepsin III', 0.0)]))])

In [ ]:
etl.fromtsv(breakpoints_fn).convertnumbers()

In [85]:
tbl_breakpoints = etl.fromtsv(breakpoints_fn).convertnumbers()
tbl_breakpoints

chrom,first_breakpoint_start,first_breakpoint_end,second_breakpoint_start,second_breakpoint_end,start_gene_id,end_gene_id,breakpoint_name
Pf3D7_14_v3,946696,946717,964505,964540,PF3D7_1407800,PF3D7_0523100,18kb around MDR1
Pf3D7_14_v3,283034,283069,300493,300522,PF3D7_1407800,PF3D7_1409200,17kb from plasmepsin IV
Pf3D7_14_v3,283034,283069,362990,363020,PF3D7_1407800,PF3D7_1409200,80kb from plasmepsin IV


# Analysis

In [70]:
tbl_pf_60_bams = etl.fromtsv(pf_6_0_bams_fn)
tbl_pf_60_bams

path,sample
/lustre/scratch116/malaria/pfalciparum/output/4/4/f/e/42104/4_bam_mark_duplicates_v2/pe.1.markdup.bam,PL0006-C
/lustre/scratch116/malaria/pfalciparum/output/3/f/1/3/45100/4_bam_mark_duplicates_v2/pe.1.markdup.bam,WS0081-C
/lustre/scratch116/malaria/pfalciparum/output/5/4/3/5/44258/4_bam_mark_duplicates_v2/pe.1.markdup.bam,QJ0018-C
/lustre/scratch116/malaria/pfalciparum/output/d/6/0/c/41088/4_bam_mark_duplicates_v2/pe.1.markdup.bam,PH0458-CW
/lustre/scratch116/malaria/pfalciparum/output/3/9/6/1/38157/4_bam_mark_duplicates_v2/pe.1.markdup.bam,PD0986-C


In [74]:
manifest_fn = '/nfs/team112_internal/rp7/data/Pf/hrp/metadata/hrp_manifest_20160621.txt'
tbl_hrp_bams = etl.fromtsv(manifest_fn)

## Identify MDR1 breakpoints by looking at samples from Rob's all_HMM_mdr1_pm23_final.txt file in IGV

In [82]:
# 
tbl_hrp_bams.selectin('sample', ['PH0254-C', 'PH0267-C', 'PH0233-C', 'PC0080-C', 'QC0156-C', 'PD0943-C', 'PG0116-C',
                                 'PD1184-C', 'PD1204-C']).displayall()

path,sample,oxford_src_code,oxford_donor_code,dataset,type,region_code,country_code,country_name,sub_continent,sub_continent_name,sub_continent_number,bases_of_5X_coverage,mean_coverage,bases_of_1X_coverage,bases_of_10X_coverage,bases_of_20X_coverage
/lustre/scratch109/malaria/pfalciparum/output/4/d/8/3/43032/1_bam_merge/pe.1.bam,PC0080-C,8388,8388,pf_community_5_0,dna,KE,KE,Kenya,EAF,East Africa,3,21446471,185.08,22440287,20675365,19603646
/lustre/scratch109/malaria/pfalciparum/output/8/8/2/9/45620/1_bam_merge/pe.1.bam,QC0156-C,MKT46,MKT46,pf_community_5_0,dna,SMM,MM,Myanmar,WSEA,Western south-east Asia,4,22631913,86.90,22841902,22452206,22046570
/lustre/scratch109/malaria/pfalciparum/output/b/5/6/3/246974/1_bam_merge/pe.1.bam,PD0943-C,NHP-04036,NHP-04036,pf_community_5_0,dna,WTH,TH,Thailand,WSEA,Western south-east Asia,4,21776395,30.73,22882928,19875972,15649296
/lustre/scratch109/malaria/pfalciparum/output/6/0/8/4/247209/1_bam_merge/pe.1.bam,PD1184-C,NHP-04863,NHP-04863,pf_community_5_0,dna,WTH,TH,Thailand,WSEA,Western south-east Asia,4,22378342,117.42,22821247,21845274,20734051
/lustre/scratch109/malaria/pfalciparum/output/c/8/5/4/247229/1_bam_merge/pe.1.bam,PD1204-C,NHP-04885,NHP-04885,pf_community_5_0,dna,WTH,TH,Thailand,WSEA,Western south-east Asia,4,21967261,87.71,22740254,20890460,18712071
/lustre/scratch109/malaria/pfalciparum/output/d/a/7/3/44267/1_bam_merge/pe.1.bam,PH0233-C,ANL2,ANL2,pf_community_5_0,dna,WKH,KH,Cambodia,ESEA,Eastern south-east Asia,5,22241945,31.80,22913701,21309507,17902706
/lustre/scratch109/malaria/pfalciparum/output/f/c/3/e/44288/1_bam_merge/pe.1.bam,PH0254-C,PF949_LP,PF949_LP,pf_community_5_0,dna,WKH,KH,Cambodia,ESEA,Eastern south-east Asia,5,22680518,82.68,22955284,22461797,21957452
/lustre/scratch109/malaria/pfalciparum/output/4/f/4/3/44301/1_bam_merge/pe.1.bam,PH0267-C,PF962_LP,PF962_LP,pf_community_5_0,dna,WKH,KH,Cambodia,ESEA,Eastern south-east Asia,5,22468780,33.30,22910900,21794246,19121689
/lustre/scratch109/malaria/pfalciparum/output/d/b/4/1/43868/1_bam_merge/pe.1.bam,PG0116-C,FCR-3,FCR-3,pf_community_5_1,dna,Lab,Lab,,Lab,Lab strains,9,22734768,94.53,22967399,22579079,22301525


In [89]:
print("mkdir -p %s/bams" % scratch_dir)
for rec in tbl_hrp_bams.selectin('sample', ['PH0254-C', 'PH0267-C', 'PH0233-C', 'PC0080-C', 'QC0156-C', 'PD0943-C', 'PG0116-C',
                                 'PD1184-C', 'PD1204-C']).data():
    original_bam = rec[0]
    macbook_bam = "%s/bams/%s.bam" % (scratch_dir, rec[1])
    print("scp malsrv2:%s %s" % (original_bam.replace('.bam', '.bam.bai'), macbook_bam.replace('.bam', '.bam.bai')))
    print("scp malsrv2:%s %s" % (original_bam, macbook_bam))


mkdir -p /lustre/scratch109/malaria/rp7/data/methods-dev/analysis/20160926_pf_60_duplications/bams
scp malsrv2:/lustre/scratch109/malaria/pfalciparum/output/4/d/8/3/43032/1_bam_merge/pe.1.bam.bai /lustre/scratch109/malaria/rp7/data/methods-dev/analysis/20160926_pf_60_duplications/bams/PC0080-C.bam.bai
scp malsrv2:/lustre/scratch109/malaria/pfalciparum/output/4/d/8/3/43032/1_bam_merge/pe.1.bam /lustre/scratch109/malaria/rp7/data/methods-dev/analysis/20160926_pf_60_duplications/bams/PC0080-C.bam
scp malsrv2:/lustre/scratch109/malaria/pfalciparum/output/8/8/2/9/45620/1_bam_merge/pe.1.bam.bai /lustre/scratch109/malaria/rp7/data/methods-dev/analysis/20160926_pf_60_duplications/bams/QC0156-C.bam.bai
scp malsrv2:/lustre/scratch109/malaria/pfalciparum/output/8/8/2/9/45620/1_bam_merge/pe.1.bam /lustre/scratch109/malaria/rp7/data/methods-dev/analysis/20160926_pf_60_duplications/bams/QC0156-C.bam
scp malsrv2:/lustre/scratch109/malaria/pfalciparum/output/b/5/6/3/246974/1_bam_merge/pe.1.bam.bai /lu

In [139]:
# Samples with more than one duplication
for rec in tbl_hrp_bams.selectin('sample', ['PF0077-C', 'PD0032-C', 'PH0247-C']).data():
    original_bam = rec[0]
    macbook_bam = "%s/bams/%s.bam" % (scratch_dir, rec[1])
    print("scp malsrv2:%s %s" % (original_bam.replace('.bam', '.bam.bai'), macbook_bam.replace('.bam', '.bam.bai')))
    print("scp malsrv2:%s %s" % (original_bam, macbook_bam))


scp malsrv2:/lustre/scratch109/malaria/pfalciparum/output/2/e/f/6/43382/1_bam_merge/pe.1.bam.bai /lustre/scratch109/malaria/rp7/data/methods-dev/analysis/20160926_pf_60_duplications/bams/PF0077-C.bam.bai
scp malsrv2:/lustre/scratch109/malaria/pfalciparum/output/2/e/f/6/43382/1_bam_merge/pe.1.bam /lustre/scratch109/malaria/rp7/data/methods-dev/analysis/20160926_pf_60_duplications/bams/PF0077-C.bam
scp malsrv2:/lustre/scratch109/malaria/pfalciparum/output/0/b/e/1/43073/1_bam_merge/pe.1.bam.bai /lustre/scratch109/malaria/rp7/data/methods-dev/analysis/20160926_pf_60_duplications/bams/PD0032-C.bam.bai
scp malsrv2:/lustre/scratch109/malaria/pfalciparum/output/0/b/e/1/43073/1_bam_merge/pe.1.bam /lustre/scratch109/malaria/rp7/data/methods-dev/analysis/20160926_pf_60_duplications/bams/PD0032-C.bam
scp malsrv2:/lustre/scratch109/malaria/pfalciparum/output/3/4/d/d/44281/1_bam_merge/pe.1.bam.bai /lustre/scratch109/malaria/rp7/data/methods-dev/analysis/20160926_pf_60_duplications/bams/PH0247-C.bam.

In [97]:
print("mkdir -p %s/bams" % scratch_dir)
for rec in tbl_hrp_bams.selectin('sample', ['PF0364-C', 'QP0184-C']).data():
    original_bam = rec[0]
    macbook_bam = "%s/bams/%s.bam" % (scratch_dir, rec[1])
    print("scp malsrv2:%s %s" % (original_bam.replace('.bam', '.bam.bai'), macbook_bam.replace('.bam', '.bam.bai')))
    print("scp malsrv2:%s %s" % (original_bam, macbook_bam))


mkdir -p /lustre/scratch109/malaria/rp7/data/methods-dev/analysis/20160926_pf_60_duplications/bams
scp malsrv2:/lustre/scratch109/malaria/pfalciparum/output/b/e/2/1/218023/1_bam_merge/pe.1.bam.bai /lustre/scratch109/malaria/rp7/data/methods-dev/analysis/20160926_pf_60_duplications/bams/QP0184-C.bam.bai
scp malsrv2:/lustre/scratch109/malaria/pfalciparum/output/b/e/2/1/218023/1_bam_merge/pe.1.bam /lustre/scratch109/malaria/rp7/data/methods-dev/analysis/20160926_pf_60_duplications/bams/QP0184-C.bam
scp malsrv2:/lustre/scratch109/malaria/pfalciparum/output/3/9/b/e/43612/1_bam_merge/pe.1.bam.bai /lustre/scratch109/malaria/rp7/data/methods-dev/analysis/20160926_pf_60_duplications/bams/PF0364-C.bam.bai
scp malsrv2:/lustre/scratch109/malaria/pfalciparum/output/3/9/b/e/43612/1_bam_merge/pe.1.bam /lustre/scratch109/malaria/rp7/data/methods-dev/analysis/20160926_pf_60_duplications/bams/PF0364-C.bam


In [79]:
tbl_hrp_bams

path,sample,oxford_src_code,oxford_donor_code,dataset,type,region_code,country_code,country_name,sub_continent,sub_continent_name,sub_continent_number,bases_of_5X_coverage,mean_coverage,bases_of_1X_coverage,bases_of_10X_coverage,bases_of_20X_coverage
/lustre/scratch109/malaria/pfalciparum/output/a/1/d/0/265637/1_bam_merge/pe.1.bam,QT0001-C,CM001-J0,CM001,pf_community_5_1,dna,BJ,BJ,Benin,WAF,West Africa,1,22429396,110.27,22866093,21829039,20453074
/lustre/scratch109/malaria/pfalciparum/output/f/1/2/c/265638/1_bam_merge/pe.1.bam,QT0001-CW4,CM001-H8,CM001,pf_community_5_1,MDA,BJ,BJ,Benin,WAF,West Africa,1,21661068,96.55,22734327,20463738,18329425
/lustre/scratch109/malaria/pfalciparum/output/0/2/0/6/265639/1_bam_merge/pe.1.bam,QT0001-CW5,CM001-H16,CM001,pf_community_5_1,MDA,BJ,BJ,Benin,WAF,West Africa,1,21409846,68.65,22651241,19897257,17210983
/lustre/scratch109/malaria/pfalciparum/output/d/a/a/f/265640/1_bam_merge/pe.1.bam,QT0001-CW6,CM001-H24,CM001,pf_community_5_1,MDA,BJ,BJ,Benin,WAF,West Africa,1,6065338,4.34,16902776,1391631,106667
/lustre/scratch109/malaria/pfalciparum/output/6/7/b/4/265641/1_bam_merge/pe.1.bam,QT0002-C,CM002-J0,CM002,pf_community_5_1,dna,BJ,BJ,Benin,WAF,West Africa,1,22434258,92.75,22854818,21789730,20157016


In [142]:
# Note the pf 6.0 bams didn't appear to have index files at the time of writing, so reverted to HRP bams
# for rec in tbl_pf_60_bams.data():
fo = open(dup_results_hrp_list_fn, 'w')
print("sample\t%s" % '\t'.join(etl.fromtsv(tandem_dup_breakpoints_fn).values('breakpoint_name')), end='\t', file=fo)
print('\t'.join(etl.fromtsv(dup_trpinv_dup_breakpoints_fn).values('breakpoint_name')), end='\t', file=fo)

i = 0
for rec in tbl_hrp_bams.data():
    i += 1
    if i%100 == 0:
        print('.', end="")
#     if(os.path.exists(rec[0]) and os.path.exists(rec[0].replace('.bam', '.bai'))):
    if(os.path.exists(rec[0]) and os.path.exists(rec[0].replace('.bam', '.bam.bai'))):
#     print(rec[0])
        duplication_genotypes = genotype_duplications(rec[0])
        print("\n%s" % rec[1], end="", file=fo)
        for dup in duplication_genotypes['faceaway']:
            print("\t%6.4f" % duplication_genotypes['faceaway'][dup], end="", file=fo)
        for dup in duplication_genotypes['dup_trpinv_dup']:
            print("\t%6.4f" % duplication_genotypes['dup_trpinv_dup'][dup], end="", file=fo)
    

...............................................................................

In [163]:
# pf 6.0 bams later got indexed, so ran on these
fo = open(dup_results_6_0_fn, 'w')
print("sample\t%s" % '\t'.join(etl.fromtsv(tandem_dup_breakpoints_fn).values('breakpoint_name')), end='\t', file=fo)
print('\t'.join(etl.fromtsv(dup_trpinv_dup_breakpoints_fn).values('breakpoint_name')), end='\t', file=fo)

i = 0
for rec in tbl_pf_60_bams.data():
    i += 1
    if i%100 == 0:
        print('.', end="")
    if(os.path.exists(rec[0]) and os.path.exists(rec[0].replace('.bam', '.bam.bai'))):
        duplication_genotypes = genotype_duplications(rec[0])
        print("\n%s" % rec[1], end="", file=fo)
        for dup in duplication_genotypes['faceaway']:
            print("\t%6.4f" % duplication_genotypes['faceaway'][dup], end="", file=fo)
        for dup in duplication_genotypes['dup_trpinv_dup']:
            print("\t%6.4f" % duplication_genotypes['dup_trpinv_dup'][dup], end="", file=fo)

fo.close()
    

..................................................................................

In [154]:
fo.close()

In [153]:
len(tbl_pf_60_bams.data())

8251

In [56]:
# PH0243-C, PfCP 5.0
bam_fn = '/lustre/scratch109/malaria/pfalciparum/output/8/c/f/3/61719/1_bam_merge/pe.1.bam'
print('\n5.0, 80kb')
print(calc_proportion_faceaway(bam_fn))
print('\n5.0, 17kb')
print(calc_proportion_faceaway(bam_fn, end_region=[300193, 300493]))
bam_fn = '/lustre/scratch116/malaria/pfalciparum/output/9/5/9/5/27793/2_gatk_print_reads_gatk3_v2/6297_1_nonhuman_7.pe.markdup.recal.bam'
print('\n6.0, 80kb')
print(calc_proportion_faceaway(bam_fn))
print('\n6.0, 17kb')
print(calc_proportion_faceaway(bam_fn, end_region=[300193, 300493]))



5.0, 80kb
(0.0, 0, 769)

5.0, 17kb
(0.09622886866059818, 74, 769)

6.0, 80kb
(0.0, 0, 779)

6.0, 17kb
(0.1245186136071887, 97, 779)
